In [ ]:
#AI Trainer for the Adjustments Project


In [26]:
#Load input data
import pandas as pd
import numpy as np

#Load data
df=pd.read_csv('SampleData.csv')
df.head(10)

,AdjustmentType,AdjustmentName,AdjustedField,Drawn,Undrawn,PD,CustomerID,CustomerType,RiskCategory,IsDefaulted,CountryCode,NewVal
0,Amend,GRC_EndMonth,RiskCategory,727232,748121,0.12,CUST450896,SME,Medium,1,GB,Medium
1,Amend,Fix_Broken_Batch,CustomerType,617646,396342,0.71,CUST359147,Retail,Low,1,GB,Retail
2,Amend,Fix_Broken_Batch,CountryCode,25383,980338,0.49,CUST563201,Corporate,Medium,0,US,DE
3,Amend,Fix_Broken_Batch,RiskCategory,217640,28801,0.54,CUST981372,Corporate,Low,0,DE,Low
4,Amend,GRC_EndMonth,IsDefaulted,950796,801071,0.50,CUST665428,SME,Medium,1,DE,0
5,Amend,GRC_EndMonth,CustomerID,734216,366855,0.00,CUST712035,SME,High,1,GB,CUST238947
6,Amend,GRC_EndMonth,RiskCategory,589877,812864,0.58,CUST238947,Retail,Low,0,DE,High
7,Amend,GRC_EndMonth,CustomerID,527559,746509,0.20,CUST192654,Retail,Low,0,GB,CUST450896
8,Amend,Defaulting_Reclassify,RiskCategory,834443,986900,0.92,CUST450896,Retail,Low,0,GB,High
9,Amend,Default,IsDefaulted,566399,640410,0.10,CUST238947,Retail,Low,0,DE,0


In [74]:
#Stage 1: OneHot encode string labels so we can use them in the model

#Build list of all string columns in the dataframe
string_columns=[]
for col in df.columns:
    if df[col].dtype=='object':
        string_columns.append(col)

#Turn output labels into numbers
outputLabels=df["NewVal"].unique()
#Create a dictionary to map the output labels to numbers
outputLabelsDict={}
for i in range(len(outputLabels)):
    outputLabelsDict[outputLabels[i]]=i
df["NewVal"]=df["NewVal"].map(outputLabelsDict)

df_encoded=pd.get_dummies(df, columns=string_columns)

#Move NewVal to the end of the dataframe
df_encoded=df_encoded[[c for c in df_encoded if c not in ['NewVal']] + ['NewVal']]

df_encoded


,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,CustomerType_Corporate,CustomerType_Retail,CustomerType_SME,RiskCategory_High,RiskCategory_Low,RiskCategory_Medium,CountryCode_DE,CountryCode_GB,CountryCode_US,NewVal
0,727232,748121,0.12,1,1,0,0,0,0,1,...,0,0,1,0,0,1,0,1,0,0
1,617646,396342,0.71,1,1,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,1
2,25383,980338,0.49,0,1,0,0,0,1,0,...,1,0,0,0,0,1,0,0,1,2
3,217640,28801,0.54,0,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,3
4,950796,801071,0.50,1,1,0,0,0,0,1,...,0,0,1,0,0,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,318438,473192,0.94,1,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,15
996,786830,25495,0.61,0,1,0,0,0,0,1,...,0,0,1,1,0,0,0,1,0,10
997,71999,152908,0.27,1,1,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,11
998,16019,509389,0.30,1,1,0,0,0,0,1,...,0,0,1,0,1,0,1,0,0,6


In [78]:
#Stage 2: Normalize all numeric values
#Normalize the dataframe

#Normalize Drawn (-1 to 1)
df_normalized=df_encoded.copy()
df_normalized["Drawn"]=(df_encoded["Drawn"]-df_encoded["Drawn"].min())/(df_encoded["Drawn"].max()-df_encoded["Drawn"].min())*2-1
#Normalize Undrawn (-1 to 1)
df_normalized["Undrawn"]=(df_encoded["Undrawn"]-df_encoded["Undrawn"].min())/(df_encoded["Undrawn"].max()-df_encoded["Undrawn"].min())*2-1

#Clip PD to 0-1
df_normalized["PD"]=df_normalized["PD"].clip(0,1)


df_normalized

,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,CustomerType_Corporate,CustomerType_Retail,CustomerType_SME,RiskCategory_High,RiskCategory_Low,RiskCategory_Medium,CountryCode_DE,CountryCode_GB,CountryCode_US,NewVal
0,0.457108,0.497036,0.12,1,1,0,0,0,0,1,...,0,0,1,0,0,1,0,1,0,0
1,0.237189,-0.206909,0.71,1,1,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,1
2,-0.951375,0.961725,0.49,0,1,0,0,0,1,0,...,1,0,0,0,0,1,0,0,1,2
3,-0.565550,-0.942394,0.54,0,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,3
4,0.905760,0.602994,0.50,1,1,0,0,0,0,1,...,0,0,1,0,0,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.363267,-0.053124,0.94,1,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,15
996,0.576710,-0.949010,0.61,0,1,0,0,0,0,1,...,0,0,1,1,0,0,0,1,0,10
997,-0.857825,-0.694044,0.27,1,1,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,11
998,-0.970167,0.019310,0.30,1,1,0,0,0,0,1,...,0,0,1,0,1,0,1,0,0,6


In [31]:
# #Split the data into test and training sets using pandas
# msk = np.random.rand(len(df_normalized)) < 0.8
# train = df_normalized[msk]
# test = df_normalized[~msk]


In [85]:
#Turn the pandas dataframe into a numpy array, split out the label and then break into sets
import tensorflow as tf

#turn df_normalized into a normal array
data=df_normalized.values

#Get all columns except the output column
X=data[:,1:]
#Get the output column
y=data[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [87]:
X_train.shape[1]

34

In [104]:
#Time to build the model

#Simple feed-forward model. No U-nets or anything fancy
ann = tf.keras.models.Sequential()

neurons=100

#Input and first hidden layer
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))

#Output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#Compile!
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



In [105]:
#Train the MODEL!!!!!
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
25/25 [==============================] - 0s 792us/step - loss: -54.1387 - accuracy: 0.0737
Epoch 2/100
25/25 [==============================] - 0s 793us/step - loss: -574.3606 - accuracy: 0.0737
Epoch 3/100
25/25 [==============================] - 0s 939us/step - loss: -3496.2783 - accuracy: 0.0737
Epoch 4/100
25/25 [==============================] - 0s 877us/step - loss: -15273.4785 - accuracy: 0.0737
Epoch 5/100
25/25 [==============================] - 0s 751us/step - loss: -49430.3438 - accuracy: 0.0737
Epoch 6/100
25/25 [==============================] - 0s 730us/step - loss: -128704.6562 - accuracy: 0.0737
Epoch 7/100
25/25 [==============================] - 0s 772us/step - loss: -286056.0625 - accuracy: 0.0737
Epoch 8/100
25/25 [==============================] - 0s 709us/step - loss: -579695.9375 - accuracy: 0.0737
Epoch 9/100
25/25 [==============================] - 0s 689us/step - loss: -1044450.6250 - accuracy: 0.0737
Epoch 10/100
25/25 [===========================